# QuantileRegression

Computes quantile regression fits over a time series, a list of numbers or a list of numeric pairs

## Documentation

### Usage

`quantile_regression(data, knots, probs)`    
     does quantile regression over the times series or data array data using the knots specification knots for the probabilities probs.

`quantile_regression(data, knots, probs, opts)`    
     does quantile regression with the options opts.

### Details & Options

- `quantile_regression` works on numpy arrays, lists of numbers, and lists of numeric pairs.

- The curves computed with quantile regression are called **regression quantiles**.

- The regression quantiles corresponding to the specified probabilities are linear combinations of B-splines generated over the specified knots.

- In other words, `quantile_regression` computes fits using a B-spline functions basis. The basis is specified with the knots argument and the option [InterpolationOrder](https://reference.wolfram.com/language/ref/InterpolationOrder).

- `quantile_regression` takes the named arguments option `order` which is 3 by default. 


## Examples

### Basic Examples

Make a random signal:

In [1]:
import numpy as np

np.random.seed(23)
n = 200
rand_data = np.column_stack((np.arange(1, n + 1), np.random.uniform(0, 100, n)))

Compute `quantile_regression` with five knots for the probabilities $0.25$ and $0.75$:

In [2]:
from QuantileRegression import *
q_funcs = quantile_regression(rand_data, knots = 5, probs = [0.25, 0.75])

Here are the formulas of the obtained regression quantiles:

In [3]:
import plotly.graph_objects as go

def qr_plot(data, q_funcs, probs, width=800, height=600):
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=data[:, 0], y=data[:, 1], mode="markers", name="data"))
    
    fig.update_layout(
        xaxis_title='X Axis Label',
        yaxis_title='Y Axis Label',
        template='plotly_dark',
        width=width,
        height=height
    )
    
    # Plot each regression quantile
    for i, q in enumerate(probs):
        y_fit = [q_funcs[i](xi) for xi in data[:, 0]]
        fig.add_trace(go.Scatter(x=data[:, 0], y=y_fit, mode='lines', name=f'{q}'))

    return fig

qr_plot(rand_data, q_funcs, probs = [0.25, 0.75]).show()

Here is a plot of the original data and the obtained regression quantiles:

Find the fraction of the data points that are under the second regression quantile:

In [4]:
len([x for x in rand_data if x[1] < q_funcs[1](x[0])]) / len(rand_data)

0.75

The obtained fraction is close to the second probability, $0.75$, given to `quantile_regression`.

### Scope

Here is a quantile regression computation over a numerical vector:

In [5]:
vec_data = np.sin(np.arange(1, 201) / 6) + np.random.uniform(-0.5, 0.5, 200)
q_funcs = quantile_regression(vec_data, knots = 12, probs = 0.5)

vec_data2 = np.column_stack((np.arange(len(vec_data)), vec_data))
qr_plot(data = vec_data2, q_funcs = q_funcs, probs = [0.5,]).show()

The second argument—the knots specification—can be an integer specifying the number of knots or a list of numbers specifying the knots of the B-spline basis:

In [27]:
qFuncs = quantile_regression(data=rand_data, knots=rand_data[::3, 0], probs=0.5)
qFuncs[0](100)

69.5048709580442

### Options

#### `order` (interpolation order)

The option `order` specifies the polynomial order of the B-spline basis. Its values are expected to be non-negative integers:

In [ ]:
q_funcs = [quantile_regression(rand_data, knots = 5, probs = 0.5, order = i)[0] for i in [0, 1, 3]]
qr_plot(data = rand_data, q_funcs = q_funcs, probs = [0, 1, 3]).show()

### Applications

#### Fit for heteroscedastic data

Here is heteroscedastic data (the variance is not constant with respect to the predictor variable):

In [13]:
x_values = np.arange(-3, 3.01, 0.01)
dist_data = np.array([
    (x, np.exp(-x**2) + np.random.normal(0, 0.15 * np.sqrt(abs(1.5 - x) / 1.5)))
    for x in x_values
])

fig = go.Figure(data=go.Scatter(x=dist_data[:, 0], y=dist_data[:, 1], mode='markers'))
fig.update_layout(
    title='Detailed Plot of distData',
    xaxis_title='x',
    yaxis_title='y',
    template='plotly_dark',
    width=800,
    height=600
)
fig.show()

Find quantile regression fits:

In [ ]:
probs = [0.1, 0.25, 0.5, 0.75, 0.9]
q_funcs = quantile_regression(dist_data, knots=4, probs=probs)

Plot the data and the regression quantiles:

In [ ]:
qr_plot(data = dist_data, q_funcs = q_funcs, probs = probs).show()

Note that the regression quantiles clearly outline the heteroscedastic nature of the data.

#### Find variance anomalies

A certain contextual type of anomaly is a subset of points that have variance very different than other subsets. Using quantile regression we can (1) evaluate the regressor-dependent variance for each point using the regression quantiles $0.25$ and $0.75$; and (2) find the points that have outlier variances.

Here we compute and plot the variance estimates for a signal:

In [ ]:
q_funcs = quantile_regression(dist_data, knots=4, probs=[0.25, 0.75])

variances = np.abs([q_funcs[1](x) - q_funcs[0](x) for x in dist_data[:, 0]])

fig = go.Figure(data=go.Scatter(y=variances, mode='markers'))
fig.update_layout(
    title='Plot of Variances',
    xaxis_title='Index',
    yaxis_title='Variance',
    template='plotly_dark',
    width=800,
    height=600
)
fig.show()

Find the lower and upper thresholds for the variance outliers:

In [ ]:
median_distances = variances - np.median(variances)
lower_threshold, upper_threshold = np.quantile(median_distances, [0.2, 0.8])
(lower_threshold, upper_threshold)

(-0.08599294135673308, 0.09164183342848714)

Find the outlier positions:

In [ ]:
variance_outlier_positions = np.where((median_distances < lower_threshold) | (median_distances > upper_threshold))[0]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dist_data[:, 0], y=dist_data[:, 1], mode='markers', name='data'))
fig.add_trace(go.Scatter(x=dist_data[variance_outlier_positions, 0], y=dist_data[variance_outlier_positions, 1], mode='markers', name='variance outliers'))

fig.update_layout(
    title='Plot of distData with Variance Outliers',
    xaxis_title='x',
    yaxis_title='y',
    template='plotly_dark',
    width=800,
    height=600
)
fig.show()

#### Fit and anomalies for financial time series

Here is a financial time series:

In [59]:
fin_data = [[3660854400, 236.16], [3662150400, 219.088], [3663360000, 217.166], [3664656000, 225.625], [3665865600, 232.085], [3667075200, 232.007], [3668371200, 244.466], [3669580800, 238.544], [3670790400, 237.622], [3672000000, 231.392], [3673209600, 230.854], [3674505600, 231.008], [3675715200, 229.316], [3677184000, 247.695], [3678393600, 243.312], [3679603200, 240.466], [3680812800, 240.851], [3682108800, 238.774], [3683318400, 228.162], [3684528000, 226.855], [3685737600, 222.857], [3686947200, 222.087], [3688156800, 236.468], [3689452800, 236.545], [3690662400, 242.235], [3691958400, 243.85], [3693254400, 241.158], [3694723200, 230.393], [3695932800, 231.008], [3697228800, 229.239], [3698438400, 227.163], [3699648000, 227.778], [3700857600, 231.008], [3702153600, 225.01], [3703363200, 220.703], [3704572800, 214.013], [3705868800, 212.167], [3707078400, 211.86], [3708374400, 201.094], [3709584000, 199.248], [3710793600, 198.249], [3712003200, 188.79], [3713212800, 193.326], [3714681600, 188.098], [3715891200, 188.943], [3717100800, 179.638], [3718310400, 156.953], [3719520000, 146.264], [3720816000, 141.573], [3722025600, 137.728], [3723321600, 133.652], [3724617600, 146.264], [3725913600, 124.04], [3727123200, 114.889], [3728592000, 112.658], [3729801600, 116.119], [3731011200, 99.124], [3732307200, 100.354], [3733516800, 112.89], [3734726400, 109.737], [3735936000, 117.58], [3737232000, 104.892], [3738441600, 99.0472], [3739737600, 103.277], [3740947200, 105.584], [3742156800, 101.278], [3743366400, 94.5872], [3744576000, 98.2016], [3745872000, 97.5096], [3747081600, 86.82], [3748291200, 94.7408], [3749500800, 86.8968], [3750710400, 65.98], [3752179200, 58.2904], [3753475200, 51.9848], [3754771200, 56.8288], [3756067200, 68.7488], [3757363200, 70.4408], [3758572800, 75.4392], [3760041600, 83.2056], [3761251200, 79.2], [3762460800, 79.04], [3763670400, 75.92], [3764966400, 74.64], [3766176000, 80.88], [3767385600, 79.68], [3768681600, 79.12], [3769891200, 82.72], [3771100800, 84.88], [3772396800, 80.48], [3773606400, 80.64], [3774816000, 64.08], [3776025600, 64.88], [3777321600, 74.72], [3778531200, 72.32], [3779740800, 70.4]]

fin_data = np.array(fin_data)

fin_data.shape

(96, 2)

Do a quantile regression fit and plot it:

In [60]:
q_funcs = quantile_regression(data = fin_data, knots=50, probs=0.5)
q_funcs

[<function QuantileRegression.QuantileRegression._make_combined_function.<locals>.<lambda>(x)>]

In [62]:
q_funcs = quantile_regression(data = fin_data, knots=50, probs=0.5)
qr_plot(data = fin_data, q_funcs=q_funcs, probs=[0.5,])

Here are the errors of the fit found:

Find anomalies’ positions in the list of fit errors:

In [ ]:

(*{688, 689, 691, 734, 736, 798}*)

Plot the data, fit and anomalies:

#### Analyze temperature time series

Get temperature data:

In [ ]:
tempData = WeatherData[{"Orlando", "Florida", "USA"}, "Temperature", {{2016, 1, 1}, {2019, 1, 1}, "Day"}]

![0t063gymqyfd5](img/0t063gymqyfd5.png)

Convert the time series into a list of numeric pairs:

In [ ]:
tempData = QuantityMagnitude[tempData["Path"]];

Compute quantile regression fits:

In [ ]:
probs = Sort[Join[Range[0.1, 0.9, 0.1], {0.01, 0.99}]];
qFuncs = QuantileRegression[tempData, 10, probs];

Plot the data and the regression quantiles:

![1c2zui0mojsp2](img/1c2zui0mojsp2.png)

Find an estimate of the conditional cumulative distribution function (![045o7e6kcme1l](img/045o7e6kcme1l.png)) at the date 2017-10-01:

![196e6ymca5g5s](img/196e6ymca5g5s.png)

Find outliers in the temperature data—outliers are defined as points below or above the $0.02$ and $0.98$ regression quantiles respectively:

In [ ]:
qFuncs = QuantileRegression[tempData, 10, {0.02, 0.98}];

In [ ]:
bottomOutliers = Select[tempData, #[[2]] < qFuncs[[1]][#[[1]]] &];
topOutliers = Select[tempData, #[[2]] > qFuncs[[-1]][#[[1]]] &];

![1avt3o921gxlv](img/1avt3o921gxlv.png)

### Properties and Relations

`quantile_regression` can be compared with ![09me7vwsq27ht](img/09me7vwsq27ht.png), ![0dpgfexdoq9tk](img/0dpgfexdoq9tk.png), ![13ergfg5oqnrj](img/13ergfg5oqnrj.png) and ![0ybzadiucxjfu](img/0ybzadiucxjfu.png):

In [ ]:
qFunc = First@QuantileRegression[distData, 4, 0.5];

In [ ]:
fFunc = LinearModelFit[distData, Table[x^i, {i, 0, 6, 1}], x]["Function"];

In [ ]:
ListLinePlot[{distData, {#, qFunc[#]} & /@ distData[[All, 1]], {#, fFunc[#]} & /@ distData[[All, 1]]}, PlotStyle -> {Thin, Thick, Thick}, PlotLegends -> {"data", 0.5`, "mean"}, PlotTheme -> "Detailed"]

![0v3533kk76ekc](img/0v3533kk76ekc.png)

Quantile regression is much more robust than linear regression. In order to demonstrate that, add a few large outliers in the data:

In [ ]:
distData3 = SortBy[Join[distData, {{1.5, 10}, {2.2, 30}}], First];

Here quantile regression and linear regression are applied, as in the previous example:

In [ ]:
qFunc = First@QuantileRegression[distData3, 4, 0.5];

In [ ]:
fFunc = LinearModelFit[distData3, Table[x^i, {i, 0, 6, 1}], x]["Function"];

Here is a plot of the obtained curves. Note that the curve corresponding to linear regression is different and a worse fit than the one from the previous example:

In [ ]:
ListLinePlot[{distData3, {#, qFunc[#]} & /@ distData3[[All, 1]], {#, fFunc[#]} & /@ distData3[[All, 1]]}, PlotStyle -> {Thin, Thick, Thick}, PlotLegends -> {"data", 0.5`, "mean"}, PlotTheme -> "Detailed"]

### Possible Issues

#### Slow computations

Because of the linear programming formulation for some data and knots specifications, the computations can be slow.

#### Fitting for probabilities 0 and 1

For most data, the quantile regression fitting for probabilities $0$ and $1$ produces regression quantiles that are "too far away from the data."

Find regression quantiles for probabilities $0$ and $0.5$ and plot them:

In [21]:
probs = [0, 0.5]
q_funcs = quantile_regression(dist_data, 6, probs)
qr_plot(data=dist_data, q_funcs=q_funcs, probs=probs).show()

In [22]:
probs = [0.5, 1]
q_funcs = quantile_regression(dist_data, 6, probs)
qr_plot(data=dist_data, q_funcs=q_funcs, probs=probs).show()

#### Overfitting

Consider the following nonlinear data:

In [8]:
nl_data = [
    [0, 2.52], [1, 2.83], [2, 3], [3, 3.2], [4.1, 3.35], [5, 3.47], [6, 3.57], [7, 3.66], [8, 3.76], [8.5, 3.81], 
    [9, 3.85], [9.5, 3.89], [10.1, 3.94], [10.5, 3.98], [11, 4.01], [11.5, 4.06], [12, 4.09], [12.5, 4.15], [13, 4.19], 
    [13.5, 4.25], [14, 4.3], [14.5, 4.35], [15, 4.41], [15.6, 4.47], [16, 4.53], [16.5, 4.6], [17, 4.68], [17.5, 4.77], 
    [18, 4.85], [18.5, 4.96], [19, 5.11], [19.55, 5.34], [19.7, 5.44], [19.9, 5.58], [20.1, 5.91], [20.3, 6.27], [20.5, 7.14], 
    [20.6, 7.14], [20.8, 7.81], [20.9, 8.32], [21, 7.75], [21.2, 9.07], [21.4, 9.49], [21.5, 9.71], [21.6, 9.83], [21.8, 10], 
    [22, 10.18], [22.1, 10.21], [22.2, 10.25], [22.3, 10.27], [22.5, 10.3], [22.7, 10.42], [22.9, 10.47], [23.1, 10.52], 
    [23.3, 10.59], [23.5, 10.63], [23.7, 10.67], [24, 10.74], [24.2, 10.78], [24.4, 10.8], [24.6, 10.82], [24.8, 10.84], 
    [25, 10.87]
]

nl_data = np.array(nl_data)

fig = go.Figure(data=go.Scatter(x=nl_data[:, 0], y=nl_data[:, 1], mode='markers'))
fig.update_layout(
    title='nlData Plot',
    xaxis_title='x',
    yaxis_title='y',
    template='plotly_dark',
    width=800,
    height=600
)
fig.show()

Make a quantile regression fit with 20 knots:

In [9]:
qFunc20 = quantile_regression(nl_data, knots=20, probs = 0.5, order = 2)[0]

Make a quantile regression fit with 40 knots:

In [10]:
qFunc40 = quantile_regression(nl_data, knots=40, probs = 0.5, order = 2)[0]

Plot the regression quantiles and the data:

In [11]:
qr_plot(nl_data, q_funcs=[qFunc20, qFunc40], probs=[20, 40]).show()

You can see that the regression quantile computed with 40 knots is "overfitted" between 0 and 8—the B-spline basis knots are too densely placed between 0 and 8.

#### Intersecting regression quantiles

When regression quantiles are overfitted, then the estimate of the conditional cumulative distribution function (CDF) can be problematic—the estimated CDF is not a monotonically increasing function.

Compute regression quantiles using “too many” knots:

In [33]:
probs = sorted(np.concatenate((np.arange(0.1, 1, 0.1), [0.01, 0.99])))
q_funcs = quantile_regression(dist_data, 30, probs)
qr_plot(data=dist_data, q_funcs=q_funcs, probs=probs, width=1200, height=400).show()

Plot the regression quantiles:finData2 = N@{10532750, 1342440, 714972, 1289802, 1302765, 814231, 830680, 416649, 80017, 24343, 1808, 1939, 25848, 32532, 21807, 26108, 12581, 36315, 34544, 3827, 7631, 7259};


Here is the estimated conditional CDF:

In [ ]:
ListLinePlot[Transpose[{Through[qFuncs[tempData[[100, 1]]]], probs}], PlotTheme -> "Detailed", FrameLabel -> {"Temperature", "Probability"}]

#### Rescaling

For certain data, it is beneficial to rescale the predictor values, predicted values, or both before doing the quantile regression computations:

In [38]:
finData2 = [10532750, 1342440, 714972, 1289802, 1302765, 814231, 830680, 416649, 80017, 24343, 1808, 1939, 25848, 32532, 21807, 26108, 12581, 36315, 34544, 3827, 7631, 7259]

In [41]:
probs = [0.2, 0.5, 0.8]
q_funcs = quantile_regression(finData2, 4, probs)
[f(10) for f in q_funcs]

[-149111.06428218874, 131492.853948295, 135002.949117364]

In [44]:
xMin = np.min(finData2)
xMax = np.max(finData2)

finData2r = [ (x - xMin) / (xMax - xMin) for x in finData2]
q_funcs = quantile_regression(finData2r, 4, probs)
[f(10) for f in q_funcs]

[-0.014330998638785362, 0.012314605071850088, 0.012647961513543922]

In [45]:
finData2r2 =np.column_stack((np.arange(len(finData2r)), finData2r))
qr_plot(finData2r2, q_funcs, probs)

### Neat Examples

Compute and plot regression quantiles over symmetric data: